<a href="https://colab.research.google.com/github/CommitAndQuit/Kvaesitso/blob/main/MACD_vectorbt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install -q vectorbt yfinance

In [26]:
nifty_50_stocks = [
    "ADANIENT.NS",
    "ASIANPAINT.NS",
    "AXISBANK.NS",
    "BAJAJ-AUTO.NS",
    "BAJFINANCE.NS",
    "BAJAJFINSV.NS",
    "BHARTIARTL.NS",
    "BPCL.NS",
    "BRITANNIA.NS",
    "CIPLA.NS",
    "COALINDIA.NS",
    "DIVISLAB.NS",
    "DRREDDY.NS",
    "EICHERMOT.NS",
    "GRASIM.NS",
    "HCLTECH.NS",
    "HDFC.NS",
    "HDFCBANK.NS",
    "HDFCLIFE.NS",
    "HEROMOTOCO.NS",
    "HINDALCO.NS",
    "HINDUNILVR.NS",
    "ICICIBANK.NS",
    "ICICILOMB.NS",
    "ICICIPRULI.NS",
    "INDUSINDBK.NS",
    "INFY.NS",
    "JSWSTEEL.NS",
    "KOTAKBANK.NS",
    "LT.NS",
    "M&M.NS",
    "MARUTI.NS",
    "NESTLEIND.NS",
    "NTPC.NS",
    "ONGC.NS",
    "POWERGRID.NS",
    "RELIANCE.NS",
    "SBICARD.NS",
    "SBILIFE.NS",
    "SBIN.NS",
    "SHREECEM.NS",
    "SUNPHARMA.NS",
    "TATACONSUM.NS",
    "TATAMOTORS.NS",
    "TATASTEEL.NS",
    "TCS.NS",
    "TECHM.NS",
    "TITAN.NS",
    "ULTRACEMCO.NS",
    "WIPRO.NS"
]
PLT_HEIGHT = 800
PLT_WIDTH = 800
DEFAULT_CASH = 10000
DEFAULT_FEES = 0.001
DEFAULT_FREQ = '1D'

In [27]:
# Configure Jupyter to display plots inline
# %matplotlib inline

import pandas as pd
import numpy as np
import yfinance as yf
import vectorbt as vbt
# import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from abc import ABC, abstractmethod

class TradingStrategy(ABC):
    """Abstract base class for trading strategies."""

    @abstractmethod
    def generate_signals(self, prices):
        """Generate entry and exit signals based on price data."""
        pass

    @abstractmethod
    def get_name(self):
        """Return the name of the strategy."""
        pass

    def run_backtest(self, prices, init_cash=DEFAULT_CASH, fees=DEFAULT_FEES, freq = DEFAULT_FREQ):
        """Run backtest using the generated signals."""
        entries, exits = self.generate_signals(prices)
        portfolio = vbt.Portfolio.from_signals(
            prices,
            entries,
            exits,
            init_cash=init_cash,
            fees=fees,
            freq=freq
        )
        return portfolio


class MACDStrategy(TradingStrategy):
    """MACD crossover strategy implementation."""

    def __init__(self, fast_window=12, slow_window=26, signal_window=9):
        self.fast_window = fast_window
        self.slow_window = slow_window
        self.signal_window = signal_window

    def get_name(self):
        return f"MACD({self.fast_window},{self.slow_window},{self.signal_window})"

    def calculate_macd(self, prices):
        """Calculate MACD indicator for the given prices."""
        return vbt.indicators.MACD.run(
            prices,
            fast_window=self.fast_window,
            slow_window=self.slow_window,
            signal_window=self.signal_window
        )

    def generate_signals(self, prices):
        """Generate entry and exit signals based on MACD crossovers."""
        macd = self.calculate_macd(prices)

        # Get MACD and signal lines
        macd_line = macd.macd
        signal_line = macd.signal

        # Calculate the difference between MACD and signal
        diff = macd_line - signal_line
        diff_shifted = diff.shift(1)

        # Entry signal: When MACD crosses above signal (diff changes from <= 0 to > 0)
        entries = (diff_shifted <= 0) & (diff > 0)

        # Exit signal: When MACD crosses below signal (diff changes from >= 0 to < 0)
        exits = (diff_shifted >= 0) & (diff < 0)

        return entries, exits


In [28]:
class MarketDataProvider:
    """Class for fetching and managing market data."""

    @staticmethod
    def fetch_data(tickers, start_date, end_date):
        """Fetch market data for the given tickers and date range."""
        start_str = start_date.strftime('%Y-%m-%d')
        end_str = end_date.strftime('%Y-%m-%d')

        data = yf.download(tickers, start=start_str, end=end_str)
        close_prices = data['Close']

        return close_prices

In [29]:

class ResultsAnalyzer:
    """Class for analyzing and presenting backtest results."""

    @staticmethod
    def create_summary(results):
        """Create a summary dataframe of key metrics."""
        metrics = ['final_value', 'total_return', 'sharpe_ratio', 'max_drawdown']
        summary = pd.DataFrame(index=results.keys(), columns=metrics)

        for ticker, res in results.items():
            for metric in metrics:
                if metric == 'total_return':
                    summary.loc[ticker, metric] = f"{res[metric] * 100:.2f}"
                elif metric == 'max_drawdown':
                    summary.loc[ticker, metric] = f"{res[metric] * 100:.2f}%"
                elif metric == 'win_rate' and 'win_rate' in res:
                    summary.loc[ticker, metric] = f"{res[metric] * 100:.2f}%"
                else:
                    summary.loc[ticker, metric] = f"{res[metric]:.2f}"

        return summary

    @staticmethod
    def get_detailed_stats(results):
        """Extract detailed statistics for each ticker."""
        detailed_stats = {}

        for ticker, res in results.items():
            portfolio = res['portfolio']
            stats = portfolio.stats()
            detailed_stats[ticker] = stats

        return detailed_stats

    @staticmethod
    def print_detailed_stats(detailed_stats):
        """Print detailed statistics for each ticker."""
        for ticker, stats in detailed_stats.items():
            print(f"\nDetailed Stats for {ticker}:")
            # Print key stats
            print(f"portfolio object: ${stats}")

In [30]:

class BacktestRunner:
    """Main class to orchestrate the backtesting process."""

    def __init__(self, tickers, start_date, end_date, strategy):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.strategy = strategy
        self.results = {}
        self.close_prices = None

    def run(self):
        """Run the backtest for all tickers."""
        try:
            # Fetch market data
            self.close_prices = MarketDataProvider.fetch_data(
                self.tickers, self.start_date, self.end_date
            )

            print(f"Available tickers in data: {self.close_prices.columns}")

            # Run backtest for each ticker
            for ticker in self.tickers:
                if ticker not in self.close_prices.columns:
                    print(f"Warning: {ticker} not found in the data. Skipping.")
                    continue

                # Run the strategy
                portfolio = self.strategy.run_backtest(self.close_prices[ticker])

                # Store results
                self.results[ticker] = {
                    'final_value': portfolio.final_value(),
                    'total_return': portfolio.total_return(),
                    'sharpe_ratio': portfolio.sharpe_ratio(),
                    'max_drawdown': portfolio.max_drawdown(),
                    'trades': portfolio.trades.records_readable,
                    'portfolio': portfolio
                }

            return self.results, self.close_prices

        except Exception as e:
            print(f"An error occurred during backtest: {e}")
            return {}, None

    def analyze_results(self):
        """Analyze and visualize the backtest results."""
        if not self.results:
            print("No results to analyze. Run the backtest first.")
            return

        # Create summary
        summary = ResultsAnalyzer.create_summary(self.results)
        print(f"{self.strategy.get_name()} Strategy Backtest Results:")
        print("Available columns in summary:", summary.columns)
        print(summary.sort_values(by="total_return", ascending=False))

        # Visualize results
        try:
            VisualizationEngine.plot_performance(self.results, self.close_prices, self.strategy)
            VisualizationEngine.compare_performance(self.results, self.close_prices, self.strategy.get_name())
        except Exception as viz_error:
            print(f"Error during visualization: {viz_error}")
            print("Continuing with statistical analysis...")

        # Get and print detailed stats
        detailed_stats = ResultsAnalyzer.get_detailed_stats(self.results)
        ResultsAnalyzer.print_detailed_stats(detailed_stats)


In [31]:
import plotly.graph_objects as go
import plotly.express as px

class VisualizationEngine:
    """Class for generating visualizations of backtest results using Plotly."""

    @staticmethod
    def plot_performance(results, close_prices, strategy):
        """Plot performance charts for each ticker using Plotly."""
        for ticker in results.keys():
            portfolio = results[ticker]['portfolio']

            # Create equity curve plot
            fig = portfolio.plot()
            fig.update_layout(title=f"{ticker} - Portfolio Value", xaxis_title="Date", yaxis_title="Portfolio Value", height=PLT_HEIGHT, width=PLT_WIDTH)
            fig.show()

            # Calculate and plot MACD
            # macd = strategy.calculate_macd(close_prices[ticker])

            # macd_fig = go.Figure()
            # macd_fig.add_trace(go.Scatter(x=close_prices[ticker].index, y=macd.macd, mode='lines', name='MACD Line'))
            # macd_fig.add_trace(go.Scatter(x=close_prices[ticker].index, y=macd.signal, mode='lines', name='Signal Line'))
            # macd_fig.update_layout(title=f"{ticker} - MACD Indicator", xaxis_title="Date", yaxis_title="MACD Value", height=PLT_HEIGHT, width=PLT_WIDTH)
            # macd_fig.show()

    @staticmethod
    def compare_performance(results, close_prices, strategy_name):
        """Compare strategy performance against buy and hold using Plotly."""
        cum_returns = pd.DataFrame(index=close_prices.index)

        for ticker in results.keys():
            cum_returns[f"{ticker}_{strategy_name}"] = results[ticker]['portfolio'].returns().cumsum()
            initial_price = close_prices[ticker].iloc[0]
            cum_returns[f"{ticker}_BuyHold"] = (close_prices[ticker] / initial_price) - 1

        # Convert to long format for Plotly
        cum_returns_long = cum_returns.reset_index().melt(id_vars=['Date'], var_name='Strategy', value_name='Cumulative Return')

        # Create interactive line plot
        fig = px.line(cum_returns_long, x='Date', y='Cumulative Return', color='Strategy',
                      title=f"Cumulative Returns: {strategy_name} Strategy vs Buy & Hold")
        fig.update_layout(xaxis_title="Date", yaxis_title="Cumulative Return", legend_title="Legend")
        fig.show()


In [32]:
if __name__ == "__main__":
    # Define the timeframe for backtesting (last 6 months)
    end_date = datetime.now() - timedelta(days=90)
    start_date = end_date - timedelta(days=180)  # 6 months

    # Define parameter grid for fast and slow windows
    fast_windows = [8, 12, 16]
    slow_windows = [17, 26, 34]

    for fast_window, slow_window in zip(fast_windows, slow_windows):
        print(f"Running backtest with fast_window={fast_window}, slow_window={slow_window}")

        # Create a MACD strategy instance
        macd_strategy = MACDStrategy(fast_window=fast_window, slow_window=slow_window, signal_window=9)

        # Create and run the backtest
        backtest = BacktestRunner(nifty_50_stocks, start_date, end_date, macd_strategy)
        backtest.run()
        backtest.analyze_results()

[**                     4%                       ]  2 of 50 completed

Running backtest with fast_window=8, slow_window=17


[*********************100%***********************]  50 of 50 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['ICICILOMB.NS', 'HDFC.NS']: YFTzMissingError('possibly delisted; no timezone found')


Available tickers in data: Index(['ADANIENT.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS',
       'BAJAJFINSV.NS', 'BAJFINANCE.NS', 'BHARTIARTL.NS', 'BPCL.NS',
       'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS',
       'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFC.NS', 'HDFCBANK.NS',
       'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS',
       'ICICIBANK.NS', 'ICICILOMB.NS', 'ICICIPRULI.NS', 'INDUSINDBK.NS',
       'INFY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS',
       'MARUTI.NS', 'NESTLEIND.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS',
       'RELIANCE.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SHREECEM.NS',
       'SUNPHARMA.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS',
       'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'ULTRACEMCO.NS', 'WIPRO.NS'],
      dtype='object', name='Ticker')


[****                   8%                       ]  4 of 50 completed

...

Available tickers in data: Index(['ADANIENT.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS',
       'BAJAJFINSV.NS', 'BAJFINANCE.NS', 'BHARTIARTL.NS', 'BPCL.NS',
       'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS',
       'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFC.NS', 'HDFCBANK.NS',
       'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS',
       'ICICIBANK.NS', 'ICICILOMB.NS', 'ICICIPRULI.NS', 'INDUSINDBK.NS',
       'INFY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS',
       'MARUTI.NS', 'NESTLEIND.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS',
       'RELIANCE.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SHREECEM.NS',
       'SUNPHARMA.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS',
       'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'ULTRACEMCO.NS', 'WIPRO.NS'],
      dtype='object', name='Ticker')


[*****                 10%                       ]  5 of 50 completed

MACD(12,26,9) Strategy Backtest Results:
Available columns in summary: Index(['final_value', 'total_return', 'sharpe_ratio', 'max_drawdown'], dtype='object')
              final_value total_return sharpe_ratio max_drawdown
BAJFINANCE.NS    10993.63         9.94         1.84       -6.52%
HCLTECH.NS       10868.83         8.69         1.64       -5.84%
BHARTIARTL.NS    10857.46         8.57         1.69       -8.61%
SBICARD.NS       10839.77         8.40         1.46       -7.56%
JSWSTEEL.NS      10552.13         5.52         1.06       -8.05%
HINDUNILVR.NS    10363.49         3.63         0.81       -7.12%
TITAN.NS         10226.42         2.26         0.52       -6.91%
HINDALCO.NS      10200.27         2.00         0.47       -7.54%
HEROMOTOCO.NS    11252.11        12.52         2.28       -7.70%
BAJAJ-AUTO.NS    11018.42        10.18         1.58      -13.84%
DIVISLAB.NS      10194.89         1.95         0.40       -8.70%
GRASIM.NS        10191.32         1.91         0.55       -4.2

[*********************100%***********************]  50 of 50 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['ICICILOMB.NS', 'HDFC.NS']: YFTzMissingError('possibly delisted; no timezone found')


Available tickers in data: Index(['ADANIENT.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS',
       'BAJAJFINSV.NS', 'BAJFINANCE.NS', 'BHARTIARTL.NS', 'BPCL.NS',
       'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS',
       'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFC.NS', 'HDFCBANK.NS',
       'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS',
       'ICICIBANK.NS', 'ICICILOMB.NS', 'ICICIPRULI.NS', 'INDUSINDBK.NS',
       'INFY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS',
       'MARUTI.NS', 'NESTLEIND.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS',
       'RELIANCE.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SHREECEM.NS',
       'SUNPHARMA.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS',
       'TCS.NS', 'TECHM.NS', 'TITAN.NS', 'ULTRACEMCO.NS', 'WIPRO.NS'],
      dtype='object', name='Ticker')
MACD(16,34,9) Strategy Backtest Results:
Available columns in summary: Index(['final_value', 'total_return', 'sharpe_ratio', 'max_draw